<a href="https://colab.research.google.com/github/cagBRT/PySpark/blob/master/Distrib_vs_Single3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook we compare doing a task in a distributed environment and on a single CPU (single computer)

In [1]:
!git clone https://github.com/cagBRT/PySpark.git

Cloning into 'PySpark'...
remote: Enumerating objects: 417, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 417 (delta 54), reused 0 (delta 0), pack-reused 330
Receiving objects: 100% (417/417), 3.37 MiB | 10.96 MiB/s, done.
Resolving deltas: 100% (252/252), done.




---



---



# Setup Spark<br>

If using DataBricks this section will be different

In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=d610fc1d803bcc4306e73f9489fc21e4bec37de5684af992181da9ef720a9cf0
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [3]:
#Import SparkSession
from pyspark.sql import SparkSession
# Create a Spark Session
#getOrCreate gets or creates a session
spark = SparkSession.builder.master("local[*]").getOrCreate()
# Check Spark Session Information
spark

In [4]:
import urllib.request
from pyspark.sql import SparkSession
from pyspark import SparkContext

In [5]:
spark = (
    SparkSession.builder
        .appName('learn')
        .config('spark.driver.memory', '8g')
        .master('local[4]',)
        .config('spark.sql.execution.arrow.pyspark.enabled', True)
        .config('spark.sql.execution.arrow.pyspark.fallback.enabled', False)
        .getOrCreate()
)

In [6]:
sc=spark.sparkContext.getOrCreate()



---



---



We will now get the Gettysburg address (a very small file)  and perform count and sort tasks.

# Get the Gettysburg address

In [7]:
import time



---

**Gettyburg Address**

---



**Count all the words using a distributed environment**

In [8]:
start_time = time.time()
Lincoln = sc.textFile("/content/PySpark/GettysBurg.txt")
LincolnCount=Lincoln.flatMap(lambda line: line.split(" ")).map(lambda word: (word, 1))
dist_lincoln_count=LincolnCount.count()
print("--- %s seconds ---" % (time.time() - start_time))
dc_time=time.time() - start_time
dist_lincoln_count

--- 6.4249818325042725 seconds ---


266

**Count all the words using a single CPU**

In [9]:
start_time = time.time()
f= open("/content/PySpark/GettysBurg.txt", "r")
single_lincoln_count= f.read()
f.close()
single_lincoln_count=single_lincoln_count.split(" ")
print("--- %s seconds ---" % (time.time() - start_time))
sc_time=time.time() - start_time
len(single_lincoln_count)

--- 0.00047588348388671875 seconds ---


263

In [10]:
print("distributed time:    ",dc_time )
print("single compute time: ", sc_time)

distributed time:     6.4252636432647705
single compute time:  0.015099048614501953


**Count unique words using a distributed environment**

In [11]:
#Count the number of distinct words
start_time = time.time()
DistinctWordsCount=LincolnCount.reduceByKey(lambda a,b: a+b)
DistinctWordsCount.count()
dc_time=time.time() - start_time
print("--- %s seconds ---" % (time.time() - start_time))
DistinctWordsCount.count()

--- 3.864929676055908 seconds ---


158

**Count the unique words using a single computer**

In [12]:
start_time = time.time()
words_unique=set(single_lincoln_count)
print("--- %s seconds ---" % (time.time() - start_time))
sc_time=time.time() - start_time
len(words_unique)

--- 0.00018644332885742188 seconds ---


155

In [13]:
print("distributed time:    ",dc_time )
print("single compute time: ", sc_time)

distributed time:     3.8586268424987793
single compute time:  0.0030274391174316406


**Count the word frequency using a distributed environment**

In [14]:
start_time = time.time()
#Sort the words by most-to-least words
SortedWordsCount=DistinctWordsCount.map(lambda a: (a[1], a[0])).sortByKey()
print("--- %s seconds ---" % (time.time() - start_time))
dc_time=time.time() - start_time
#print most frequent 20 words
#SortedWordsCount.top(20)

--- 1.0817790031433105 seconds ---


**Count the word frequency using a single computer**

In [16]:
start_time = time.time()
counted_words=[]
words_unique=list(words_unique)
for i in range(len(words_unique)):
  tuples= (words_unique.count(words_unique[i]), words_unique[i])
  counted_words.append(tuples)
print("--- %s seconds ---" % (time.time() - start_time))
sc_time= time.time() - start_time


--- 0.0006151199340820312 seconds ---


In [17]:
print("distributed time:    ",dc_time )
print("single compute time: ", sc_time)

distributed time:     1.082019329071045
single compute time:  0.0008859634399414062
